In [1]:
import torch
import sklearn
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
train = pd.read_csv("dataset/random_instruction_train.csv")

In [3]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,instruction_a,instruction_b
0,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,B,Prompt: What is the difference between marriag...,Prompt: What is the difference between marriag...
1,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,C,explain function calling. how would you call a...,explain function calling. how would you call a...
2,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,A,How can I create a test set for a very rare ca...,How can I create a test set for a very rare ca...
3,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,B,What is the best way to travel from Tel-Aviv t...,What is the best way to travel from Tel-Aviv t...
4,292873,vicuna-13b,gpt-4-0314,"[""Construct a rap battle, in the style of Epic...","[""[Zeus]\nYo, it's the king of the gods on the...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...",0,1,0,B,"Construct a rap battle, in the style of Epic R...","Construct a rap battle, in the style of Epic R..."
...,...,...,...,...,...,...,...,...,...,...,...,...
54459,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,A,"A simple mnemonic for \u03c0:\n\""How I wish I ...","A simple mnemonic for \u03c0:\n\""How I wish I ..."
54460,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0,A,"In python, implement a naive Bayes with gaussi...","In python, implement a naive Bayes with gaussi..."
54461,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0,A,Prompt: is it unethical to work on building we...,Prompt: is it unethical to work on building we...
54462,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0,B,"If a bait contains 0,0025% bromadiolon then ho...","If a bait contains 0,0025% bromadiolon then ho..."


In [6]:
train.prompt.values[0]

'["What is the difference between marriage license and marriage certificate?","How can I get both of them as quick as possible in California ","What is the minimal time to get them? 1 day or 10 days?"]'

In [33]:
train.loc[724,'prompt']

'["relate philosophy to mathematics","Structure","when should structure be prioritized over concept","when should structure be prioritized over concept","teaching a structure, vs synthesizing the structure via specific concept learning"]'

In [64]:
output_as = []
output_bs = []
scuess_idx = []
labels = []
ids = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    try:
        prompt = row["prompt"]
        response_a = row["response_a"]
        response_b = row["response_b"]
        label = row['label']
        id = row['id']
        if prompt.find('","') != -1:
            cnt = prompt.count('","')
            prompt = row["prompt"].split('","')
            response_a = row["response_a"].split('","')
            response_b = row["response_b"].split('","')
            for sentence_idx in range(cnt + 1):
                output_a = "###Model A\n"
                output_b = "###Model B\n"
                output_a += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
                output_a += "Response: " + response_a[sentence_idx][2:].strip() + "\n\n"
                output_b += "Prompt: " + prompt[sentence_idx][2:].strip() + "\n"
                output_b += "Response: " +response_b[sentence_idx][2:].strip() + "\n\n"
                output_as.append(output_a)
                output_bs.append(output_b)
                scuess_idx.append(idx)
                labels.append(label)
                ids.append(id)
        else:
            output_a = "###Model A\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_a"][2:-2].strip() + "\n\n"
            output_b = "###Model B\nPrompt: " + row["prompt"][2:-2].strip() + "\nResponse: " + row["response_b"][2:-2].strip() + "\n\n"
            output_as.append(output_a)
            output_bs.append(output_b)
            scuess_idx.append(idx)
            labels.append(label)
            ids.append(id)
        
    except Exception as e:
        print(f"Error: {idx}")
        print(f"{e}")
        # output_as.append(" ")
        # output_bs.append(" ")

  4%|▍         | 2318/54464 [00:00<00:04, 11930.55it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

  9%|▉         | 4880/54464 [00:00<00:04, 11807.65it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

 14%|█▎        | 7468/54464 [00:00<00:04, 11540.86it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_d

###Model A
Prompt: Make a funny joke
Response: Sure, here's one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!


###Model A
Prompt: i am doing a project on twitter sentiment analysis from comment. Can you write and conclusion for me
Response: Sure, here's a sample conclusion for your Twitter sentiment analysis project:\n\nIn conclusion, our study on Twitter sentiment analysis using comments has provided valuable insights into the sentiments expressed by users on the platform. We utilized various tools, techniques, and models to analyze the data and identified various trends and patterns in the sentiment of Twitter users.\n\nOur findings suggest that Twitter users express a wide range of emotions, including positive, negative, and neutral sentiments. We also observed that the sentiment of Twitter users varies depending on the topic of the conversation, with certain topics tending to elicit more positive or negative responses from users.\n\nAdditionally

In [65]:
assert len(scuess_idx)==len(output_as)

In [66]:
new_train = pd.DataFrame({'id': ids, 'instruction_a': output_as, 'instruction_b': output_bs, 'label': labels })

In [77]:
new_train.id.value_counts()

id
2387179051    36
3479108101    29
3076453552    22
2846599172    21
2298429166    20
              ..
1620015064     1
1620143181     1
134190253      1
1620207100     1
4294947231     1
Name: count, Length: 54464, dtype: int64

In [83]:
print(new_train.loc[new_train.id==1620207100,'instruction_a'].values[0])

###Model A
Prompt: What does perplexity mean in terms of llms
Response: Perplexity is a metric used to evaluate language models in natural language processing. Specifically:\n\n- Language models assign a probability to sequences of words in a language. For example, given the sequence \"The cat sat on the...\", a language model would assign a probability to the next word being \"mat\", \"couch\", \"table\", etc.\n\n- Perplexity measures how well a language model predicts a sample of text. Intuitively, lower perplexity means the language model is good at predicting the text.\n\n- The formula for perplexity is:\n\nPerplexity = 2^(Cross Entropy)\n\nWhere cross entropy is the average negative log likelihood the language model assigns to each word.\n\n- So a lower perplexity score indicates the language model assigns higher probability to the words in the text on average. This means it is good at modeling and predicting that text.\n\n- In LLMs like GPT-3, perplexity is used to test how well 

In [76]:
new_train.id=='1620207100'

0        False
1        False
2        False
3        False
4        False
         ...  
67351    False
67352    False
67353    False
67354    False
67355    False
Name: id, Length: 67356, dtype: bool

In [5]:
# train = train.loc[scuess_idx,:].reset_index(drop = True)
# train['instruction_a'] = output_as
# train['instruction_b'] = output_bs

In [6]:
# train = train.loc[:5,:]

In [32]:
def get_instruction_prompt(row):
    r_a = row.instruction_a
    r_b = row.instruction_b
    background = "Here are two question-answering dialogues. Compare two model performance on answering question, determine which is better."
    options = "###options\nA.Model A\nB. Model B\nC.Tie"
    #instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{background}\n\n###Model A\n{r_a}###Model B\n{r_b}{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere is {background}\n\n###Model A\n{r_a}###Model B\n{r_b}{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    row['instruction_prompt'] = instruct_prompt
    return row

In [33]:
new_train = new_train.apply(lambda x: get_instruction_prompt(x), axis = 1)

In [35]:
print(new_train.loc[0,'instruction_prompt'])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Here is Here are two question-answering dialogues. Compare two model performance on answering question, determine which is better.

###Model A
Prompt: What is the difference between marriage license and marriage certificate?
Response: A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.

###Model B
Prompt: Wh

In [46]:
tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id)

'<|end_of_text|>'

In [36]:
len(tokenizer.encode(new_train.loc[0,'instruction_prompt'],add_special_tokens=True))

463

In [40]:
length = [len(tokenizer.encode(new_train.loc[i,'instruction_prompt'],add_special_tokens=True)) for i in range(len(new_train))]

In [45]:
length.index(1555)

56

In [44]:
[i for i in length if i>1500]

[1555,
 5259,
 1740,
 2884,
 1801,
 1952,
 1557,
 2245,
 1765,
 1634,
 1782,
 1617,
 1541,
 1882,
 2953,
 2556,
 1571,
 3842,
 2349,
 1937,
 1717,
 2794,
 1759,
 1553,
 10293,
 2004,
 1882,
 3720,
 1946,
 5717,
 6347,
 3260,
 5028,
 2218,
 1548,
 2074,
 1534,
 6402,
 1519,
 2652,
 2515,
 1761,
 2229,
 9843,
 2004,
 1551,
 1714,
 1571,
 1763,
 1872,
 2059,
 1680,
 6330,
 4551,
 6061,
 2055,
 3110,
 2253,
 3566,
 2006,
 1710,
 2348,
 1783,
 2568,
 1907,
 1709,
 1612,
 1517,
 2148,
 1558,
 9674,
 1766,
 2943,
 2169,
 1701,
 1501,
 1631,
 1943,
 1953,
 2730,
 2730,
 4392,
 1860,
 2548,
 2553,
 2175,
 4252,
 2795,
 5756,
 2061,
 1833,
 2272,
 2051,
 1676,
 6383,
 1821,
 3588,
 6040,
 6771,
 1563,
 6953,
 1502,
 1546,
 1525,
 3432,
 2034,
 1625,
 4954,
 2236,
 3395,
 2177,
 2027,
 5512,
 3518,
 1795,
 2006,
 1795,
 1946,
 1959,
 2017,
 1661,
 1975,
 1812,
 3151,
 2706,
 5079,
 1639,
 2635,
 1682,
 2139,
 1665,
 1579,
 2050,
 3826,
 2610,
 2681,
 1786,
 2531,
 3174,
 1587,
 1522,
 2138,
 2629

In [ ]:
new_train.loc[56,'instruction_prompt']

In [38]:
example = new_train.loc[0,'instruction_prompt']
token_ids = tokenizer.encode(example,add_special_tokens=True, truncation=True,
                                          max_length=1024)
tokens = tokenizer.convert_ids_to_tokens(token_ids)
for token, token_id in zip(tokens, token_ids):
    print(f"Token: {token}, ID: {token_id}")

Token: <|begin_of_text|>, ID: 128000
Token: <|start_header_id|>, ID: 128006
Token: user, ID: 882
Token: <|end_header_id|>, ID: 128007
Token: ĊĊ, ID: 271
Token: Here, ID: 8586
Token: Ġis, ID: 374
Token: ĠHere, ID: 5810
Token: Ġare, ID: 527
Token: Ġtwo, ID: 1403
Token: Ġquestion, ID: 3488
Token: -, ID: 12
Token: ans, ID: 598
Token: w, ID: 86
Token: ering, ID: 4776
Token: Ġdialog, ID: 7402
Token: ues, ID: 1157
Token: ., ID: 13
Token: ĠCompare, ID: 24702
Token: Ġtwo, ID: 1403
Token: Ġmodel, ID: 1646
Token: Ġperformance, ID: 5178
Token: Ġon, ID: 389
Token: Ġanswering, ID: 36864
Token: Ġquestion, ID: 3488
Token: ,, ID: 11
Token: Ġdetermine, ID: 8417
Token: Ġwhich, ID: 902
Token: Ġis, ID: 374
Token: Ġbetter, ID: 2731
Token: .ĊĊ, ID: 382
Token: ###, ID: 14711
Token: Model, ID: 1747
Token: ĠA, ID: 362
Token: Ċ, ID: 198
Token: Prompt, ID: 55715
Token: :, ID: 25
Token: ĠWhat, ID: 3639
Token: Ġis, ID: 374
Token: Ġthe, ID: 279
Token: Ġdifference, ID: 6811
Token: Ġbetween, ID: 1990
Token: Ġmarriage,

In [56]:
from torch.utils.data import Dataset
class InstructionDataSet(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        super(InstructionDataSet, self).__init__()
        #self.data = data.sample(len(data), random_state=0).reset_index(drop=True)
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        # self.A_token = self.tokenizer.encode(text='A', add_special_tokens=False, truncation=True, )
        # self.B_token = self.tokenizer.encode(text='B', add_special_tokens=False, truncation=True, )
        # self.C_token = self.tokenizer.encode(text='C', add_special_tokens=False, truncation=True, )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        now_data = self.data.loc[index]
        r_a = now_data['instruction_a']
        r_b = now_data['instruction_b']
        background = "Here are two question-answering dialogues. Compare two model performance on answering question, determine which is better."
        options = "###options\nA.Model A\nB. Model B\nC.Tie"
        instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere is {background}\n\n###Model A\n{r_a}###Model B\n{r_b}{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>A<|end_of_text|>"

        templete_part1 = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n\n"
        templete_part1_input_ids = self.tokenizer.encode(text=templete_part1, add_special_tokens=True,)
        
        model_a_input_ids = self.tokenizer.encode(text=r_a, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length // 2)
        model_b_input_ids = self.tokenizer.encode(text=r_b, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length // 2)
        templete_part2 = "###options\nA. Model A\nB. Model B\nC. Tie\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
        templete_part2_input_ids = self.tokenizer.encode(text=templete_part2, add_special_tokens=True,)
        
        label = now_data['label']
        label_ids = self.tokenizer.encode(text=label, add_special_tokens=False, truncation=True, )
        input_ids = templete_part1_input_ids + model_a_input_ids + model_b_input_ids + templete_part2_input_ids + label_ids + [self.tokenizer.eos_token_id]
        labels = [-100] * (len(input_ids) - 2) + label_ids + [self.tokenizer.eos_token_id]

        return {
            "input_ids": input_ids,
            "labels": labels
        }

In [52]:
from transformers import AutoTokenizer, AutoConfig
MODEL = "meta-llama/llama-3-transformers-8b-hf-v1"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from dataclasses import dataclass
@dataclass
class DataCollatorForInstruction:
     tokenizer: PreTrainedTokenizerBase
        model: Optional[Any] = None
        padding: Union[bool, str, PaddingStrategy] = True
        max_length: Optional[int] = None
        pad_to_multiple_of: Optional[int] = None
        label_pad_token_id: int = -100
        return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        labels = [feature["labels"] for feature in features] if "labels" in features[0].keys() else None
        # We have to pad the labels before calling `tokenizer.pad` as this method won't pad them and needs them of the
        # same length to return tensors.
        if labels is not None:
            max_label_length = max(len(l) for l in labels)
            if self.pad_to_multiple_of is not None:
                max_label_length = (
                        (max_label_length + self.pad_to_multiple_of - 1)
                        // self.pad_to_multiple_of
                        * self.pad_to_multiple_of
                )

            padding_side = self.tokenizer.padding_side
            for feature in features:
                remainder = [self.label_pad_token_id] * (max_label_length - len(feature["labels"]))
                if isinstance(feature["labels"], list):
                    feature["labels"] = (
                        feature["labels"] + remainder if padding_side == "right" else remainder + feature["labels"]
                    )
                elif padding_side == "right":
                    feature["labels"] = np.concatenate([feature["labels"], remainder]).astype(np.int64)
                else:
                    feature["labels"] = np.concatenate([remainder, feature["labels"]]).astype(np.int64)
        # breakpoint()
        features = self.tokenizer.pad(
            features,
            padding=True,
            max_length=max_label_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors,
        )

        # prepare decoder_input_ids
        if (
                labels is not None
                and self.model is not None
                and hasattr(self.model, "prepare_decoder_input_ids_from_labels")
        ):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features["labels"])
            features["decoder_input_ids"] = decoder_input_ids
        # breakpoint() # [(len(features[i]['input_ids']),len(features[i]['labels'])) for i in range(4)]
        return features

In [58]:
example = InstructionDataSet(new_train,tokenizer, 1024, 1)[1]

token_ids = example['input_ids']
tokens = tokenizer.convert_ids_to_tokens(token_ids)
for token, token_id in zip(tokens, token_ids):
    print(f"Token: {token}, ID: {token_id}")

Token: <|begin_of_text|>, ID: 128000
Token: <|start_header_id|>, ID: 128006
Token: user, ID: 882
Token: <|end_header_id|>, ID: 128007
Token: ĊĊ, ID: 271
Token: Here, ID: 8586
Token: Ġare, ID: 527
Token: Ġtwo, ID: 1403
Token: Ġquestion, ID: 3488
Token: -, ID: 12
Token: ans, ID: 598
Token: w, ID: 86
Token: ering, ID: 4776
Token: Ġdialog, ID: 7402
Token: ues, ID: 1157
Token: ., ID: 13
Token: ĠCompare, ID: 24702
Token: Ġtwo, ID: 1403
Token: Ġmodel, ID: 1646
Token: Ġperformance, ID: 5178
Token: Ġon, ID: 389
Token: Ġanswering, ID: 36864
Token: Ġquestion, ID: 3488
Token: ,, ID: 11
Token: Ġdetermine, ID: 8417
Token: Ġwhich, ID: 902
Token: Ġis, ID: 374
Token: Ġbetter, ID: 2731
Token: .ĊĊ, ID: 382
Token: ###, ID: 14711
Token: Model, ID: 1747
Token: ĠA, ID: 362
Token: Ċ, ID: 198
Token: Prompt, ID: 55715
Token: :, ID: 25
Token: Ġw, ID: 289
Token: Ġcan, ID: 649
Token: ĠI, ID: 358
Token: Ġget, ID: 636
Token: Ġboth, ID: 2225
Token: Ġof, ID: 315
Token: Ġthem, ID: 1124
Token: Ġas, ID: 439
Token: Ġquick